In [3]:
from lumibot.brokers import Alpaca
from lumibot.traders import Trader


In [4]:
ALPACA_CONFIG = {
    # Put your own Alpaca key here:
    "API_KEY": "PKJ9BFP6PBGXQFKIZ61J",
    # Put your own Alpaca secret here:
    "API_SECRET": "XbtfmUXQ1laMmoFhAPJxPAcQUweQBZoAABJSDJhg",
    # Set this to False to use a live account
    "PAPER": True
}

In [5]:
from lumibot.strategies.strategy import Strategy

class MyStrategy(Strategy):  # noqa: D101
    # Custom parameters
    parameters = {  # noqa: RUF012
        "symbol": "SPY",
        "quantity": 1,
        "side": "buy"
    }

    def initialize(self, symbol=""):  # noqa: D102
        # Will make on_trading_iteration() run every 180 minutes
        self.sleeptime = "180M"

    def on_trading_iteration(self):  # noqa: D102
        symbol = self.parameters["symbol"]
        quantity = self.parameters["quantity"]
        side = self.parameters["side"]

        order = self.create_order(symbol, quantity, side)
        self.submit_order(order)

In [6]:
trader = Trader()
broker = Alpaca(ALPACA_CONFIG)
strategy = MyStrategy(name="My Strategy", budget=10000, broker=broker, symbol="SPY")

2024-12-04 16:48:00 | asyncio | INFO | [unknown] Waiting for the socket stream connection to be established, 
                method _stream_established must be called
2024-12-04 16:48:00 | alpaca.trading.stream | INFO | started trading stream
2024-12-04 16:48:00 | alpaca.trading.stream | INFO | starting trading websocket connection
2024-12-04 16:48:01 | alpaca.trading.stream | INFO | connected to: BaseURL.TRADING_STREAM_PAPER


In [7]:
from datetime import datetime
from lumibot.backtesting import YahooDataBacktesting

backtesting_start = datetime(2000, 1, 1)
backtesting_end = datetime(2024, 12, 1)
strategy.run_backtest(
    YahooDataBacktesting,
    backtesting_start,
    backtesting_end,
    parameters={
        "symbol": "SPY"
    },
)

2024-12-04 16:48:04 | asyncio | INFO | Starting backtest...
2024-12-04 16:48:04 | lumibot.traders.trader | INFO | Backtesting starting...


Progress || 100.00%  [Elapsed: 0:05:28 ETA: 0:00:00] Portfolio Val: 8,467,335.28

({'cagr': 0.19508882985097986,
  'volatility': 6.477905189953946,
  'sharpe': 0.02335459137950833,
  'max_drawdown': {'drawdown': 1.509549861285013,
   'date': Timestamp('2009-03-06 08:30:00-0500', tz='America/New_York')},
  'romad': 0.12923642660263596,
  'total_return': 83.67335277981941},
 <__main__.MyStrategy at 0x321fe3cb0>)

In [ ]:
trader.add_strategy(strategy)
trader.run_all()